Imports

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import os
from PIL import Image
import numpy as np
import pandas as pd

In [2]:
def to_numpy(tensor):
    return tensor.cpu().numpy()
 
def maximum_weight_aggregation(models_states):
    aggregated_weights = {}
    for param_name in models_states[0]:
        all_weights = [to_numpy(model_state[param_name]) for model_state in models_states]
        aggregated_weights[param_name] = torch.tensor(np.max(all_weights, axis=0))
    return aggregated_weights, "MaximumWeight"

def minimum_weight_aggregation(models_states):
    aggregated_weights = {}
    for param_name in models_states[0]:
        all_weights = [to_numpy(model_state[param_name]) for model_state in models_states]
        aggregated_weights[param_name] = torch.tensor(np.min(all_weights, axis=0))
    return aggregated_weights, "MinimumWeight"

def mean_aggregation(models_states):
    aggregated_weights = {}
    for param_name in models_states[0]:
        all_weights = [to_numpy(model_state[param_name]) for model_state in models_states]
        aggregated_weights[param_name] = torch.tensor(np.mean(all_weights, axis=0))
    return aggregated_weights, "MeanWeight"

def median_aggregation(models_states):
    aggregated_weights = {}
    for param_name in models_states[0]:
        all_weights = [to_numpy(model_state[param_name]) for model_state in models_states]
        aggregated_weights[param_name] = torch.tensor(np.median(all_weights, axis=0))
    return aggregated_weights, "MedianWeight"

def trimmed_mean_aggregation(models_states, trim_fraction=0.2):
    aggregated_weights = {}
    for param_name in models_states[0]:
        all_weights = [to_numpy(model_state[param_name]) for model_state in models_states]
        sorted_weights = np.sort(all_weights, axis=0)
        trim_size = int(trim_fraction * len(sorted_weights))
        trimmed_weights = sorted_weights[trim_size:-trim_size]
        aggregated_weights[param_name] = torch.tensor(np.mean(trimmed_weights, axis=0))
    return aggregated_weights, "TrimmedMean"

def geometric_mean_aggregation(models_states):
    aggregated_weights = {}
    for param_name in models_states[0]:
        all_weights = [to_numpy(model_state[param_name]) for model_state in models_states]
        num_negatives = np.sum(np.array(all_weights) < 0, axis=0)
        if np.sum(num_negatives) == 0:
            aggregated_weights[param_name] = torch.tensor(np.exp(np.mean(np.log(all_weights), axis=0)))
        else:
            product = np.prod(all_weights, axis=0)
            aggregated_weights[param_name] = torch.tensor(np.sign(product) * np.abs(product)**(1/num_negatives))
        
    return aggregated_weights, "GeometricMean"

def harmonic_mean_aggregation(models_states):
    aggregated_weights = {}
    for param_name in models_states[0]:
        all_weights = [to_numpy(model_state[param_name]) for model_state in models_states]
        all_weights_np = np.array(all_weights)  # Convert the list to a NumPy array
        aggregated_weights[param_name] = torch.tensor(len(all_weights_np) / np.sum(1.0 / all_weights_np, axis=0))
    return aggregated_weights, "HarmonicMean"

def weighted_average_aggregation(models_states, model_weights):
    aggregated_weights = {}
    for param_name in models_states[0]:
        all_weights = [to_numpy(model_state[param_name]) for model_state in models_states]
        weights_array = np.array(model_weights)
        normalized_weights = weights_array / np.sum(weights_array)
        aggregated_weights[param_name] = torch.tensor(np.sum(normalized_weights * all_weights, axis=0))
    return aggregated_weights, "WeightedAverage"

In [3]:
# Define CarDataSet Class
class CarDataSet():

    # Define The Initialization
    def __init__(self, csv_file, root_dir, transform=None, target_transform=None):
        self.cars = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.target_transform = target_transform
        self.resize = transforms.Resize((150,150))  # Resize images to a uniform size

    # Define The Length Function
    def __len__(self):
        return len(self.cars)

    # Define The Get Item Function
    def __getitem__(self, idx):

        # Pull The Image And Check Settings
        img_name = os.path.join(self.root_dir, self.cars.iloc[idx, 0])

        image = Image.open(img_name)

        if image.mode != 'RGB':
          image = image.convert('RGB')

        # Pull The Label, -1 To Normalize To 0
        label = (self.cars.iloc[idx, 5]) - 1

        if self.transform:
            image = self.transform(image)

        # Define The Dictionary
        sample = {'image': image, 'cars': label}

        # Return
        return sample
    
# Test Accuracy
def test_accuracy(model, test_loader_internal, passed_device, loss_fn):

    # Set Parameters
    model.to(passed_device)
    correct = 0
    total = 0
    run = 0
    val_loss = 0

    # Run Tests
    with torch.no_grad():
        for test_data_internal in test_loader_internal:
            images_test_acc, labels_test_acc = test_data_internal['image'].cuda(), test_data_internal['cars'].cuda()
            outputs_test_acc = model(images_test_acc)
            _, predicted_test_acc = torch.max(outputs_test_acc.data, 1)
            val_loss += (loss_fn(outputs_test_acc, labels_test_acc)).item()
            total += labels_test_acc.size(0)
            correct += (predicted_test_acc == labels_test_acc).sum().item()
            run += 1

    # Return
    return (100 * correct / total), val_loss/run

In [4]:
# Define Transform
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])
    
# Load The Data
test_data = CarDataSet(csv_file='../SynchronizationProject/stanford_cars_eec174/val_make.csv',
                                        root_dir='../SynchronizationProject/stanford_cars_eec174/images/val', transform=transform)

train_data = CarDataSet(csv_file='../SynchronizationProject/stanford_cars_eec174/train_make.csv',
                                        root_dir='../SynchronizationProject/stanford_cars_eec174/images/train', transform=transform)

In [5]:
InternalSave = pd.read_csv("../SynchronizationProject/SaveData/DataFrames/InfoSave.csv")

In [6]:
InternalSave

,Run,BatchSize,LearningRate,Epochs,TrainAccuracy,TrainLoss,TrainHistory,ValAccuracy,ValLoss,ValHistory,GPUCount,GPUNumber,PreTrain,TrainTime,SavePathInput,SavePathOutput
0,0,32,0.0001,50,98.891674,0.046048,"{'loss': [3.4871723436841777, 3.42232744647007...",18.830361,5.840313,"{'loss': [3.490916973666141, 3.408613242601093...",1,1,False,6513.744140,NaN,../SynchronizationProject/SaveData/Weights/0.pth
1,1,32,0.0001,50,99.954355,0.006749,"{'loss': [3.5152197248795454, 3.43448365435880...",12.691829,5.167353,"{'loss': [3.5044365682099996, 3.46959972381591...",3,1,False,2789.132076,NaN,../SynchronizationProject/SaveData/Weights/1T0...
2,1,32,0.0001,50,99.989637,0.011231,"{'loss': [3.5432352767271156, 3.46236025024862...",11.489009,4.928459,"{'loss': [3.492495696795614, 3.476980432083732...",3,2,False,2805.588549,NaN,../SynchronizationProject/SaveData/Weights/1T1...
3,1,32,0.0001,50,99.990760,0.004735,"{'loss': [3.547464345483219, 3.464133195316090...",11.198673,5.037350,"{'loss': [3.549210316256473, 3.463777341340717...",3,3,False,2776.819473,NaN,../SynchronizationProject/SaveData/Weights/1T2...
4,2,32,0.0001,50,99.991356,0.025019,"{'loss': [3.5846809398296267, 3.42887252985045...",8.917462,5.132797,"{'loss': [3.548779399771439, 3.598160019046382...",6,1,False,1857.319781,NaN,../SynchronizationProject/SaveData/Weights/2T0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,15,64,0.0010,50,94.594299,0.204781,"{'loss': [4.1057514777550335, 3.74745035171508...",3.815844,10.712571,"{'loss': [3.54023992387872, 3.5036307259609827...",10,6,False,1980.333326,NaN,../SynchronizationProject/SaveData/Weights/15T...
76,15,64,0.0010,50,94.982639,0.172121,"{'loss': [4.1091077877924995, 3.72035032052260...",7.009540,8.698870,"{'loss': [3.5532967040413306, 3.48865530992809...",10,7,False,1936.754572,NaN,../SynchronizationProject/SaveData/Weights/15T...
77,15,64,0.0010,50,96.576995,0.137701,"{'loss': [4.296231306516207, 3.586264573610746...",6.885110,9.257041,"{'loss': [3.596015873708223, 3.476873178231089...",10,8,False,1963.545982,NaN,../SynchronizationProject/SaveData/Weights/15T...
78,15,64,0.0010,50,97.842856,0.118109,"{'loss': [4.12328287271353, 3.6071051634274998...",5.723766,8.788563,"{'loss': [3.575199949113946, 3.524588001401801...",10,9,False,1930.976732,NaN,../SynchronizationProject/SaveData/Weights/15T...


In [7]:
ProcessedData = pd.DataFrame(columns = ["Run", "BatchSize", "LearningRate", "Epochs", "TrainAccuracy", "TrainLoss", "ValAccuracy", "ValLoss", "GPUCount", "MergeType", "PreTrain", "TrainTime", "SavePathOutput"])

total_runs = InternalSave['Run'].iloc[-1]

for run_number in range(0, total_runs + 1):
    print(run_number)
    specific_run_rows = InternalSave[InternalSave['Run'] == run_number]
    learning_rate = int(specific_run_rows["LearningRate"].iloc[0])
    batch_size = int(specific_run_rows["BatchSize"].iloc[0])
    epochs = specific_run_rows["Epochs"].iloc[0]
    train_time = specific_run_rows["TrainTime"].iloc[0]
    pretrain = specific_run_rows["PreTrain"].iloc[0]
    
    
    train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)
    test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size = batch_size, shuffle = True)
    if len(specific_run_rows) == 1:
        
        # Model Definition and Final Layer Edit
        net = models.resnet50()
        net.fc = nn.Linear(net.fc.in_features, 49)
        net.load_state_dict(torch.load(list(specific_run_rows["SavePathOutput"])[0]))
        
         # Load Model Onto GPU
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        net.to(device)

        # Loss Function and Optimizer
        loss_function = nn.CrossEntropyLoss()

        trainacc, trainloss = test_accuracy(net, train_loader, device, loss_function)
        valacc, valloss = test_accuracy(net, test_loader, device, loss_function)

        PATH = '../SynchronizationProject/SaveData/ProcessedWeights/' + str(run_number) + '.pth'
        torch.save(net.state_dict(), PATH)

        new_data_row = {
            "Run": run_number,
            "BatchSize": batch_size,
            "LearningRate": learning_rate, 
            "Epochs": epochs,
            "TrainAccuracy": trainacc,
            "TrainLoss": trainloss,
            "ValAccuracy": valacc,
            "ValLoss": valloss,
            "GPUCount": 1,
            "MergeType": "NONE",
            "PreTrain": pretrain, 
            "TrainTime": train_time,
            "SavePathOutput": PATH
        }

        ProcessedData.loc[len(ProcessedData)] = new_data_row
        
    else:
        weightsList = []
        for pathindex in range(0,len(specific_run_rows)):
            weightsList.append(torch.load(list(specific_run_rows["SavePathOutput"])[pathindex]))
        
        print("Start")
        aggregatedData = []
        # print(1)
        aggregatedData.append(maximum_weight_aggregation(weightsList))
        # print(2)
        aggregatedData.append(minimum_weight_aggregation(weightsList))
        # print(3)
        aggregatedData.append(median_aggregation(weightsList))
        aggregatedData.append(mean_aggregation(weightsList))
        # print(4)
        #aggregatedData.append(trimmed_mean_aggregation(weightsList, trim_fraction=0.2)) this causes issues and also like kinda doesnt work the weay i want it to. Easy to fix, just dont think i wanna use
        # print(5)
        #aggregatedData.append(geometric_mean_aggregation(weightsList)) #signed if there are negative numbers, regular if there are not NEED TO MAYBE WORK ON THIS, SOMETHING WITH THE num_negatives CAn hasve a 0 INDICE,. AND THAT NEED TO BE HANDELED PROPERLY
        #print(6)
        #aggregatedData.append(harmonic_mean_aggregation(weightsList))
        #aggregatedData.append(weighted_average_aggregation(weightsList, model_weights))  Decide on this later    
        
        for item in aggregatedData:
            print(item[1])
            # Model Definition and Final Layer Edit
            net = models.resnet50()
            net.fc = nn.Linear(net.fc.in_features, 49)
            
            net.load_state_dict(item[0])
            
             # Load Model Onto GPU
            device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
            net.to(device)
            
            # Loss Function and Optimizer
            loss_function = nn.CrossEntropyLoss()

            trainacc, trainloss = test_accuracy(net, train_loader, device, loss_function)
            valacc, valloss = test_accuracy(net, test_loader, device, loss_function)

            PATH = '../SynchronizationProject/SaveData/ProcessedWeights/' + str(run_number) + str(item[1]) + '.pth'
            torch.save(net.state_dict(), PATH)

            new_data_row = {
                "Run": run_number, 
                "BatchSize": batch_size, 
                "LearningRate": learning_rate, 
                "Epochs": epochs, 
                "TrainAccuracy": trainacc, 
                "TrainLoss": trainloss, 
                "ValAccuracy": valacc, 
                "ValLoss": valloss, 
                "GPUCount": len(specific_run_rows), 
                "MergeType": item[1], 
                "PreTrain": pretrain, 
                "TrainTime": train_time,
                "SavePathOutput": PATH
            }

            ProcessedData.loc[len(ProcessedData)] = new_data_row

file_path = '../SynchronizationProject/SaveData/DataFrames/ProcessedData.csv'

ProcessedData.to_csv(file_path, index=False)

0
1
Start
MaximumWeight
MinimumWeight
MedianWeight
MeanWeight
2
Start
MaximumWeight
MinimumWeight
MedianWeight
MeanWeight
3
Start
MaximumWeight
MinimumWeight
MedianWeight
MeanWeight
4
5
Start
MaximumWeight
MinimumWeight
MedianWeight
MeanWeight
6
Start
MaximumWeight
MinimumWeight
MedianWeight
MeanWeight
7
Start
MaximumWeight
MinimumWeight
MedianWeight
MeanWeight
8
9
Start
MaximumWeight
MinimumWeight
MedianWeight
MeanWeight
10
Start
MaximumWeight
MinimumWeight
MedianWeight
MeanWeight
11
Start
MaximumWeight
MinimumWeight
MedianWeight
MeanWeight
12
13
Start
MaximumWeight
MinimumWeight
MedianWeight
MeanWeight
14
Start
MaximumWeight
MinimumWeight
MedianWeight
MeanWeight
15
Start
MaximumWeight
MinimumWeight
MedianWeight
MeanWeight
